## 객체 추적과 모션 벡터

1. 배경차분(background Subtraction:BS)
    - 등록된 배경 모델과 현재 입력 프레임과의 차영상을 이용하여 전경 객체를 검출
    - 움직이는 전경 객체 검출을 위한 기본적인 방법
    
2. 정적 배경 차분
    - 정적 배경을 이용한 전경 객체 검출 실행 결과
    - 배경 영상과 현재 프레임에 대해 각각 가우시안 필터링을 수행하여 잡음 제거
    
    

In [1]:
import sys 
import numpy as np
import cv2 

# 비디오 파일 열기 
cap = cv2.VideoCapture('PETS2000.avi')

if not cap.isOpened():
    print('Video open failed!')
    sys.exit()
    
# 배경 영상 등록 
ret, back = cap.read()

if not ret:
    print('Background image registration failed!')
    sys.exit()
    
back = cv2.cvtColor(back, cv2.COLOR_BGR2GRAY)
back = cv2.GaussianBlur(back, (0,0), 1.0)

# 비디오 매 프레임 처리 
while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (0,0), 1.0)
    
    # 차영상 구하기 
    diff = cv2.absdiff(gray, back)
    _, diff = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    
    cv2.imshow('frame',frame)
    cv2.imshow('diff',diff)
    
    if cv2.waitKey(30)== 27:
        break
        
    


cap.release()
cv2.destroyAllWindows()


In [ ]:
import sys
import numpy as np
import cv2 

# 비디오 파일 열기 
cap = cv2.VideoCapture('PETS2000.avi')

if not cap.isOpened():
    print('Video open failed!')
    sys.exit()
    
# 배경 영상 등록 
ret, back = cap.read()

if not ret:
    print('Background image registration failed!')
    sys.exit()
    

back = cv2.cvtColor(back, cv2.COLOR_BGR2GRAY)
back = cv2.GaussianBlur(back, (0,0), 1.0)

# 비디오 매 프레임 처리 
while True:
    ret, frame = cap.read()
    
    if not ret:
        break 
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (0,0), 1.0)
    
    # 차영상 구하기 & 이진화 
    diff = cv2.absdiff(gray, back)
    _, diff = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    
    # 레이블링을 이용하여 바운딩 박스 표시 
    cnt, _, stats, _ = cv2.connectedComponentsWithStats(diff)
    
    for i in range(1, cnt):
        x,y,w,h,s = stats[i]
        
        if s< 100:
            continue
            
        cv2.rectangle(frame, (x,y,w,h),(0,0,255),2)
        
        cv2.imshow('frame',frame)
        cv2.imshow('diff',diff)

    if cv2.waitKey(30) == 27:
        break
cap.release()
cv2.destroyAllWindows()

이동 평균 배경차분\

- 정적 배경 모델 사용시 문제점
- 미리 등록된 기준 영상이 실제 배경과 크게 달라질 경우 오동작한다
(예) 그림자 등의 영향으로 인한 조도 변경, 새로운 객체가 화면에 고정될 경우\

평균 연산에 의한 배경 영상 생성\
- 움직이는 객체가 존재하는 수백 장의 입력 영상으로부터 평균 영상을 구하면?\
=>  수백 장의 이전 프레임을 버퍼에 저장하려면 대용량 메모리가 필요

이동 평균(Moving average)이란?\
- 수백 장의 영상을 저장하는 대신 매 프레임이 들어올 때마다 평균 영상을 갱신
- 대용량 버퍼 메모리가 필요하지 않음

이동 평균 계산을 위한 가중치 누적 함수\
cv2.accumulateWeighted(src, dst, alpha, mask=None)=> dst\

src: 입력 영상, 1또는 3채널 . 8비트 호는 32 비트 실수형\
dst:축적 영상. 입력 영상과 동일 채널 개수. 32비트 또는 64비트 실수형\
alpha:(입력 영상에 대한 ) 가중치\
mask: 마스크 영상

In [1]:
import sys 
import numpy as np
import cv2 

# 비디오 파일 열기 
cap = cv2.VideoCapture('PETS2000.avi')

if not cap.isOpened():
    print('Video open failed!')
    sys.exit()
    
# 배경 영상 등록 
ret, back = cap.read()

if not ret:
    print('Background image registration failed!')
    sys.exit()
    
# back: uint8 배경, fback: float32배경

back = cv2.cvtColor(back, cv2.COLOR_BGR2GRAY)
back = cv2.GaussianBlur(back, (0,0), 1.0)
fback = back.astype(np.float32)

# 비디오 매 프레임 처리 
while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (0,0), 1.0)
    
    # fback
    cv2.accumulateWeighted(gray, fback, 0.01)
    back = fback.astype(np.uint8)
    
    
    diff = cv2.absdiff(gray, back)
    _, diff = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    
    # 레이블링을 이용하여 바운딩 박스 표시 
    cnt, _, stats, _ = cv2.connectedComponentsWithStats(diff)
    
    for i in range(1, cnt):
        x,y,w,h,s = stats[i]
        
        if s < 100:
            continue
        
        cv2.rectangle(frame, (x, y, w, h), (0,0,255), 2)
        
    cv2.imshow('frame',frame)
    cv2.imshow('diff',diff)
    cv2.imshow('back',back)
    
    if cv2.waitKey(30) == 27:
        break 
cap.release()
cv2.destroyAllWindows()